# SETR (Rethinking Semantic Segmentation from a Sequence-to-Sequence Perspective with Transformers)  
- SETR 논문 리뷰

## Abstract  
- 최근 semantic segmentation은 encoder-decoder 형태의 FCN 구조를 따른다  
- encoder는 feature의 resolution을 계속해서 줄이며 semantic information을 학습한다  
- context module은 segmentation에서 굉장히 중요하며 dilated convolution or attention module을 이용해 receptive field를 증가시키는 것에 집중했다  
- 이 논문에서는 semantic segmentation을 sequence-to-sequence problem으로 다룸으로써 대체 가능한 관점을 제공하는 것을 목표로 한다  
- transformer의 모든 레이어에서 모델링된 global context를 통해 encoder는 간단한 decoder와 결합한다  
- 이를 통해 SETR이라는 강력한 모델을 제공한다  
- SETR은 ADE20K와 Pascal Context dataset에서 SOTA를 달성하고 Cityscapes에서는 경쟁력 있는 성능을 보여준다  

## Introduction  
- FCN은 encoder-decoder 구조로 이루어져 있다  
- encoder는 이미지 이해를 위해 여러 개의 CNN으로 구성되어 있다  
- 계산 비용 때문에 feature map의 해상도는 계속해서 줄어들고 encoder는 서서히 증가하는 receptive field로 semantic information을 학습한다  
- 이러한 디자인은 두 개의 메리트가 있는데 translation equivariance와 locality이다  
- locality의 경우, parameter를 공유해 모델의 복잡성을 제어한다  
- 그러나 이는 이미지에서 segmentation에 중요한 long-range dependency information을 학습하는데 제한된 receptive field로 인해 어려워진다는 근본적인 한계를 제기한다  
- 최근 연구에서 convolution을 완전히 없애고 attention을 효율적으로 사용하는 모델이 나오고 있다  
- 이 논문에서 우리는 semantic segmentation model design에 대한 rethinking과 어떤 대안을 제공하는 것을 목표로 한다  
- 우리는 convolution 대신 순수한 transformer를 사용한 모델 SETR을 제안한다  
- 제일 먼저 이미지를 patch로 분해한 후 sequence로 형성한다  
- patch를 flatten시켜 linear embedding을 적용한 후 transformer의 input으로 들어갈 feature embedding vector를 얻는다  
- 학습된 feature를 얻으면 decoder는 original image resolution으로 upsampling한다  
- 결정적으로 downsample을 수행하지 않으며 transformer encoder의 모든 레이어에 global context modeling을 수행한다  
- 따라서 semantic segmentation에 완전히 새로운 관점을 제안하게 된다  

**Contribution**  
- 1. 우리는 segmentation을 sequence-to-sequence problem으로 재구성해 FCN의 대안을 제시했다  
- 2. 이미지를 sequentializing하여 fully attentive feature representation을 구현하기 위해 transformer를 이용한다  
- 3. 이를 살펴보기 위해 다양한 복잡성을 가진 3개의 다른 decoder design을 소개한다  

- 광범위한 연구를 통해 우리의 SETR 모델은 attention moduel이 없거나 다른 FCN 구조의 모델과 비교해 우수한 feature representation을 학습할 수 있으며 SOTA를 달성했다  

## Method  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/figure1.JPG?raw=true" width=1000>

### FCN-based semantic segmentation  
- convolution operation의 locality 특성 때문에 receptive field는 레이어의 깊이에 따라 선형적으로 증가한다  
- 결과적으로 large receptive field는 레이어가 깊을수록 long-range dependency이다  
- 그러나 특정 깊이의 레이어에 도달하게 되면 이러한 장점이 없어진다  
- context modeling에 대해 제한된 receptive field를 가지는 것은 FCN 구조의 본질적인 한계이다  
- FCN과 attention module을 결합한 형태의 최근 몇몇 모델은 long-range contextual information을 학습하는데 효율적인 방법을 제시했다  
- 이런 method는 quadratic complexity 때문에 더 깊은 레이어의 attention 학습을 제한다  
- 이는 lower-level feature에서 dependency learning이 부족하며 차선의 representation learning으로 이어진다는 것을 의미한다  
- 이러한 한계를 극복하기 위해 우리는 SETR을 제안한다  

### Segmentation transformers (SETR)  
- transformer는 $\textbf{z}\in\mathcal{R}^{L\times C}$를 input으로 받는다  
- 이때 $L$은 sequence의 길이, $C$는 channel size이다  
- 따라서 SETR도 마찬가지로 input image $\textbf{x}\in\mathcal{R}^{H\times W\times 3}$를 $\textbf{z}$로 변환해 input으로 받는다  
- 이미지 sequentialization을 하는 간단한 방법은 단순히 flatten 시키는 것이다  
- 예를 들어 $480\times 480\times 3$ 이미지를 flatten 시키면 691,200의 길이를 가지는 벡터를 얻는다  
- 그러나 이는 너무 고차원이며 계산적으로도 비효율적이다  
- 따라서 모든 픽셀에 대해 일일이 tokenizing을 하는 것은 불가능하다  
- 일반적인 encoder는 $2D$ 이미지를 downsampling한다는 것을 고려해 transformer의 input sequence 길이를 $\frac{H}{16}\times \frac{W}{16}=\frac{HW}{256}$으로 결정한다  
- $\frac{HW}{256}$의 input sequence를 얻기 위해 input image $\textbf{x}\in\mathcal{R}^{H\times W\times 3}$를 $\frac{H}{16}\times\frac{W}{16}$ patch의 grid로 균등하게 나눈 후 sequence로 flatten 시킨다  
- 더 나아가 linear projection $f: p\rightarrow e \in \mathcal{R}^C$를 사용해 벡터화된 patch p를 latent C dimensional embedding space로 매핑을 함으로써 이미지 $\textbf{x}$에 대한 patch embedding의 $1D$ sequence를 얻는다  
- patch를 encoding하기 위해 각 embedding $p_i$와 location embedding $e_i$로 형성된 $E=\left\{e_1+p_1,\cdots, e_L+p_L\right\}$를 학습한다  
- $1D$ embedding sequence가 input으로 들어오면 transformer encoder는 feature representation을 학습한다  
- 이는 transformer의 모든 레이어는 global receptive field를 가지고 있음을 의미하며 FCN에서의 문제점인 제한된 receptive field를 해결했다  
- transformer encoder는 MSA block과 MLP block으로 구성되어 있다  
- 각 레이어에서 self-attention의 query, key, value를 얻는다  
$$query = Z^{l-1}\textbf{W}_Q,\, key=Z^{l-1}\textbf{W}_K\, , value=Z^{l-1}\textbf{W}_V$$  
- 이때 $\textbf{W}_Q\,,\textbf{W}_K\,,\textbf{W}_V\in \mathcal{R}^{C\times d}$는 학습 가능한 파라미터들이고 $d$는 q, k, v의 dimension이다  
- 따라서 self-attention은 다음과 같이 계산된다  
$$SA\left(Z^{l-1}\right)=Z^{l-1}+softmax\left(\frac{Z^{l-1}\textbf{W}_Q\left(Z\textbf{W}_K\right)^T}{\sqrt{d}}\right)\left(Z^{l-1}\textbf{W}_V\right)$$  
- MSA는 $m$개의 독립적인 SA 연산을 가진 matrix이며 output은 다음과 같다  
$$MSA\left(Z^{l-1}\right)=\left[SA_1\left(Z^{l-1}\right);SA_2\left(Z^{l-1}\right);\cdots;SA_m\left(Z^{l-1}\right)\right]$$  
- 이 output은 residual skip과 MLP block 연산을 수행한다  
$$Z^l=MSA\left(Z^{l-1}\right)+MLP\left(MSA\left(Z^{l-1}\right)\right)\in\mathcal{R}^{L\times C}$$  
- Layer Norm은 MSA와 MLP block 이전에 적용된다  

### Decoder designs  
- 3개의 decoder design을 소개한다  
- decoder의 목적은 segmentation map을 생성하는 것이므로 encoder의 output $\frac{HW}{256}$을 $\frac{H}{16}\times \frac{W}{16}\times C$로 reshape을 할 필요가 있다  

**(1) Naive upsampling (Naive)**  
- naive decoder는 먼저 transformer feature를 category number로 project 시킨다  
- 이를 위해 아키텍처에 간단하게 2개의 layer를 추가한다  
- $1\times1$ conv + sync batch norm (w/ReLU) + $1\times1$ conv  
- 그 후, output을 input size에 맞춰 bilinear upsampling을 수행한다  
- 그리고 cross entropy loss를 사용한다  

**(2) Progressive UPsampling (PUP)**  
- noise prediction이 많은 one step upsampling 대신에 우리는 progressive upsampling을 고려한다  
- adversarial effect를 최대한 완화시키기 위해 upsampling은 2배까지 제한한다  
- 따라서 $\frac{H}{16}\times\frac{W}{16}\times C$ 사이즈에서 4번 upsampling을 수행해 원본 사이즈를 얻는다  

**(3) Multi-Level feature Aggregation (MLA)**  
- 세 번째는 MLA이며 이는 feature prymid network와 비슷하다  
- 그러나 이 decoder는 모든 레이어의 feature resolution이 같기 때문에 pyramid 형태와는 다르다  
- 이 decoder는 input으로 $\left\{Z^m\right\}\left(m\in\left\{\frac{L_e}{M},2\frac{L_e}{M}, \cdots,M\frac{L_e}{M}\right\}\right)$을 받는다  
- M stream은 각각 선택된 하나의 특정 레이어에 초점을 맞춰 배치된다  
- 각 stream에서 먼저 encoder의 output $\frac{HW}{256}\times C$를 $\frac{H}{16}\times\frac{W}{16}\times C$로 reshape을 한다  
- $1\times1$, $3\times3$, $3\times3$ conv이 첫 번째, 세 번째 레이어에서 각 feature 채널이 절반으로 축소된 상태에 적용되며 이후 $4\times4$ bilinear upsampling을 수행한다  

## Experiments  
### Experimental setup  

**Cityscapes**  
- 19 object categories  
- train: 2975, valid: 500, test: 1525  
- resolution: $2048\times 1024$  

**ADE20K**  
- 150 classes  
- train: 20210, valid: 2000, test: 3352  

**PASCAL Context**  
- 60 classes  
- train: 4998, valid: 5105  

**Implementation details**  
- data augmentation  
    - random resize with (0.5, 2.0)  
    - random cropping with (768, 512, 480) for Cityscapes, ADE20K and Pascal Context  
    - random horizontal flipping  

- We set batch size 16 and the total iteration to 160,000 and 80,000 for the experiments on ADE20K and Pascal Context  
- For Cityscapes, we set batch size to 8  
- We adopt a polynomial learning rate decay schedule  
- SGD momentum 0.9 with weight decay 0  
- We set initial learning rate 0.001 on ADE20K and Pascal Context, and 0.01 on Cityscapes  

**Auxiliary loss**  
- 모델을 훈련할 때 auxiliary loss를 적용한다  
- auxiliary loss는 SETR-Naive에서 $\left(Z^{10}, Z^{15}, Z^{20}\right)$, SETR-PUP에서 $\left(Z^{10}, Z^{15}, Z^{20}, Z^{24}\right)$, SETR-MLA에서 $\left(Z^6, Z^{12}, Z^{18}, Z^{24}\right)$에 적용된다  

**Multi-scale test**  
- random scaling factor는 (0.5, 0.75, 1.0, 1.25, 1.5, 1.75)로 적용한다  
- 또한 훈련의 간편화를 위해 널리 사용되는 OHEM loss는 사용하지 않는다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/table1.JPG?raw=true" width=600>

**SETR variants**  
- decoder는 3개의 변형 모델을 사용한다 (SETR-Naive, SETR-PUP, SETR-MLA)  
- encoder는 2개의 변형 모델을 사용한다 (T-Base, T-Large)  

**Pre-training**  
- ViT 또는 DeiT에서 제공하는 pre-trained weight를 사용한다  
- FCN encoder of Hybrid의 경우 ImageNet-1K의 pre-trained weight를 사용한다    
- patch size로는 $16\times 16$을 사용한다  

**Evaluation metric**  
- mean IoU를 사용한다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/table2.JPG?raw=true" width=600>
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/table3.JPG?raw=true" width=600>
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/figure5.JPG?raw=true" width=600>
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/figure6.JPG?raw=true" width=600>


### Ablation stuides  
- table 2와 3는 batch size 8로 Cityscapes에서 훈련했으며 valid set의 mIoU를 각각 비교한 표다  
- table 2에서 우리는 다음의 observation을 얻을 수 있다  
    - 1. feature map을 계속해서 upsampling하는 SETR-PUP은 Cityscapes set에서 variants 중 가장 좋은 성능을 낸다  
    - 2. 전체적으로 T-Base보다 T-Large model의 성능이 더 좋다  
    - 3. SETR-PUP-Base (76.71%)는 Hybrid-Base (76.76%)보다 성능이 좋지 않지만 iteration을 80K로 늘리면 78.02%로 더 성능이 좋아졌다  
    - 4. Pre-training은 굉장히 중요하며 random weight로 SETR-PUP을 학습할 경우 42.27%의 mIoU의 성능을 낸다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/figure2.JPG?raw=true" width=600>
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/table4.JPG?raw=true" width=600>

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/figure3.JPG?raw=true" width=600>
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/table5.JPG?raw=true" width=600>

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/figure4.JPG?raw=true" width=800>
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/table6.JPG?raw=true" width=600>
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SETR/figure/table7.JPG?raw=true" width=600>

### Comparison to state-of-the-art  

**Results on ADE20K**  
- ADE20K에서 SETR-MLA가 50.28%로 가장 좋은 성능을 낸다  

**Results on Pascal Context**  
- Pascal Context 역시 SETR-MLA가 55.83%로 가장 좋은 성능을 보여주고 있다  

**Results on Cityscapes**  
- Cityscapes valid set에서는 SETR-PUP이 82.15%로 가장 좋은 성능을 낸다  
- 그러나 test set에서는 SOTA를 달성하지 못했지만 다른 모델들과 비교해 충분히 경쟁력있는 성능을 보여주고 있다  

## Conclusion  
- 우리는 semantic segmentation을 sequence-to-sequence로 해결하려는 새로운 관점을 제안했다  
- 기존 method들의 경우, large receptive field를 얻기 위해 dilated convolution, attention module 등 다양한 방법을 시도했지만 우리는 FCN에 의존 없이 receptive filed challenge를 해결했다  
- 다양한 decoder 모델을 구현하고 최신 method 없이 우리만의 강력한 semantic segmentation 모델을 구축했다  
- 다양한 실험을 통해 ADE20K와 Pascal Context dataset에서는 SOTA를 달성하고 Cityscapes dataset에서는 충분히 경쟁력있는 성능을 보여주고 있다